In [1]:
%pip install matplotlib torch


In [2]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Parses the rollout data from demo.py and saves them as numpy arrays of states, actions, and pot handle positions (the conditional vector data)

import numpy as np
import csv
import matplotlib.pyplot as plt
import pickle as pkl
import torch
from scipy.spatial.transform import Rotation as R
from transform_utils import quat_to_rot6d, rotvec_to_rot6d, rot6d_to_quat
from image_encoder import VisionTransformerEncoder


In [ ]:
# Initialize ViT encoder for image processing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit_encoder = VisionTransformerEncoder(latent_dim=128)
vit_encoder.to(device).eval()

VisionTransformerEncoder(
  (patch_embed): ImagePatchAndEmbed(
    (projection): Linear(in_features=768, out_features=768, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): MultiHeadAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (projection): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=768, out_features=3072, bias=True)
        (1): GELU(approximate='none')
        (2): Dropout(p=0.1, inplace=False)
        (3): Linear(in_features=3072, out_features=768, bias=True)
        (4): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_fe

In [5]:
# with open("rollouts_pot/rollout_seed0_mode2.pkl", "rb") as f:
#     rollout = pkl.load(f)
#     obs = rollout["observations"]
#     actions = np.array(rollout["actions"])
#     robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
#     robot0_eef_quat = np.array([o["robot0_eef_quat"] for o in obs])
#     robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
#     robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
#     robot1_eef_quat = np.array([o["robot1_eef_quat"] for o in obs])
#     robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

#     repeats_needed = 250 - actions.shape[0]

#     repeated_last = np.tile(actions[-1], (repeats_needed, 1))
#     actions = np.vstack([actions, repeated_last])

#     repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
#     robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
#     state = robot0_eef_pos

#     repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
#     robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
#     robot0_eef_rotvec = R.from_quat(robot0_eef_quat).as_rotvec()
#     state = np.hstack([state, robot0_eef_rotvec])


#     repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
#     robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
#     robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
#     state = np.hstack([state, robot0_gripper_pos])

#     repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
#     robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
#     state = np.hstack([state, robot1_eef_pos])

#     repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
#     robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
#     robot1_eef_rotvec = R.from_quat(robot1_eef_quat).as_rotvec()
#     state = np.hstack([state, robot1_eef_rotvec])

#     repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
#     robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
#     robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
#     state = np.hstack([state, robot1_gripper_pos])

# print(np.shape(state))
# print(np.shape(actions))

In [ ]:
expert_states_list = []
expert_actions_list = []
pot_start_list = []
pot_states_list1 = []
pot_states_list2 = []
image_latents_list0 = []
image_latents_list1 = []
for i in [2, 3]:
    for j in [0]:  # Only process seed 0 since we only have those files   
        with open("rollouts/newslower/rollout_seed%s_mode%s.pkl" % (j, i), "rb") as f: #CHANGE ME! - USE SEED 100 ONLY FOR TESTING PURPOSES, AND CHANGE THE FILE NAME TO WHAT IT IS ON MACHINE.
            rollout = pkl.load(f)
            obs = rollout["observations"]
            actions = np.array(rollout["actions"])
            print(np.shape(actions))
            pot1 = np.array(rollout["pot_states1"])
            pot2 = np.array(rollout["pot_states2"])
            pot = np.array(rollout["pot_start"])

            pot_start_list.append(np.concatenate((pot[0], pot[1])))

            print("here")

            if 'camera0_obs' in rollout and 'camera1_obs' in rollout:
                camera0_latents = []
                camera1_latents = []
                for frame_idx in range(len(rollout['camera0_obs'])):
                    print(f"Processing frame {frame_idx} of {len(rollout['camera0_obs'])}")
                    img0 = torch.from_numpy(rollout['camera0_obs'][frame_idx]).permute(2,0,1).unsqueeze(0).float().to(device)
                    img1 = torch.from_numpy(rollout['camera1_obs'][frame_idx]).permute(2,0,1).unsqueeze(0).float().to(device)
                    with torch.no_grad():
                        camera0_latents.append(vit_encoder(img0).cpu().numpy().squeeze())
                        camera1_latents.append(vit_encoder(img1).cpu().numpy().squeeze())
                camera0_latents = np.array(camera0_latents)
                camera1_latents = np.array(camera1_latents)
            else:
                camera0_latents = np.zeros((len(actions), 128))
                camera1_latents = np.zeros((len(actions), 128))

            print("processing rollout")

            robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
            robot0_eef_quat = np.array([o["robot0_eef_quat_site"] for o in obs])
            robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
            robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
            robot1_eef_quat = np.array([o["robot1_eef_quat_site"] for o in obs])
            robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

            repeats_needed = 700 - actions.shape[0]

            repeated_last = np.tile(actions[-1], (repeats_needed, 1))
            actions = np.vstack([actions, repeated_last])

            repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
            robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
            state = robot0_eef_pos

            print("processing rollout")

            repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
            robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
            robot0_eef_rotvec = R.from_quat(robot0_eef_quat).as_rotvec()
            state = np.hstack([state, robot0_eef_rotvec])


            repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
            robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
            robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
            state = np.hstack([state, robot0_gripper_pos])

            repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
            robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
            state = np.hstack([state, robot1_eef_pos])

            print("processing rollout")

            repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
            robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
            robot1_eef_rotvec = R.from_quat(robot1_eef_quat).as_rotvec()
            state = np.hstack([state, robot1_eef_rotvec])

            repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
            robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
            robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
            state = np.hstack([state, robot1_gripper_pos])

            repeated_last = np.tile(pot1[-1], (repeats_needed, 1))
            pot1 = np.vstack([pot1, repeated_last])
            repeated_last = np.tile(pot2[-1], (repeats_needed, 1))
            pot2 = np.vstack([pot2, repeated_last])
            pot_states_list1.append(pot1)
            pot_states_list2.append(pot2)

            repeated_last = np.tile(camera0_latents[-1], (repeats_needed, 1))
            camera0_latents = np.vstack([camera0_latents, repeated_last])
            repeated_last = np.tile(camera1_latents[-1], (repeats_needed, 1))
            camera1_latents = np.vstack([camera1_latents, repeated_last])
            image_latents_list0.append(camera0_latents)
            image_latents_list1.append(camera1_latents)

            print("processing rollout")

            expert_states_list.append(state)
            expert_actions_list.append(actions)

            print("moving on...")

expert_states_rotvec = np.stack(expert_states_list, axis=0)
expert_actions_rotvec = np.stack(expert_actions_list, axis=0)
pot_states_rotvec1 = np.stack(pot_states_list1, axis=0)
pot_states_rotvec2 = np.stack(pot_states_list2, axis=0)
pot_start_rotvec = np.stack(pot_start_list, axis=0)
image_latents_rotvec0 = np.stack(image_latents_list0, axis=0)
image_latents_rotvec1 = np.stack(image_latents_list1, axis=0)

(649, 14)
here
Processing 649 frames with BATCH processing...
  Batch 1/82: frames 0-8
  Batch 2/82: frames 8-16
  Batch 3/82: frames 16-24
  Batch 4/82: frames 24-32


KeyboardInterrupt: 

In [ ]:
print(np.shape(expert_states_rotvec))
print(np.shape(expert_actions_rotvec))
print(np.shape(pot_states_rotvec1))
print(np.shape(pot_states_rotvec2))
print(np.shape(pot_start_rotvec))
print(np.shape(image_latents_rotvec0))
print(np.shape(image_latents_rotvec1))

(20, 700, 14)
(20, 700, 14)
(20, 700, 3)
(20, 700, 3)
(20, 6)


In [ ]:
np.save("data/expert_states_newslower_20.npy", expert_states_rotvec)
np.save("data/expert_actions_newslower_20.npy", expert_actions_rotvec)
np.save("data/pot_states1_newslower_20.npy", pot_states_rotvec1)
np.save("data/pot_states2_newslower_20.npy", pot_states_rotvec2)
np.save("data/pot_start_newslower_20.npy", pot_start_rotvec)
np.save("data/arm1_images_latents.npy", image_latents_rotvec0)
np.save("data/arm2_images_latents.npy", image_latents_rotvec1)


In [ ]:
# with open("rollouts/rollout_seed0_mode2.pkl", "rb") as f:
#     rollout = pkl.load(f)
#     obs = rollout["observations"]
#     actions = np.array(rollout["actions"])

#     pos0 = actions[:,:3]
#     rotvec0 = actions[:,3:6]
#     gripper0 = actions[:,6]
#     pos1 = actions[:,7:10]
#     rotvec1 = actions[:,10:13]
#     gripper1 = actions[:,13]

#     rot6d_list0 = []
#     for rv in rotvec0:
#         rot6d_list0.append(rotvec_to_rot6d(rv))
#     rot6d0 = np.array(rot6d_list0)

#     rot6d_list1 = []
#     for rv in rotvec1:
#         rot6d_list1.append(rotvec_to_rot6d(rv))
#     rot6d1 = np.array(rot6d_list1)

#     actions = np.concatenate((pos0, rot6d0, gripper0.reshape(-1, 1), pos1, rot6d1, gripper1.reshape(-1, 1)), axis=1)

#     robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
#     robot0_eef_quat = np.array([o["robot0_eef_quat"] for o in obs])
#     robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
#     robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
#     robot1_eef_quat = np.array([o["robot1_eef_quat"] for o in obs])
#     robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

#     repeats_needed = 250 - actions.shape[0]

#     repeated_last = np.tile(actions[-1], (repeats_needed, 1))
#     actions = np.vstack([actions, repeated_last])

#     repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
#     robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
#     state = robot0_eef_pos

#     repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
#     robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
#     eef_rot6d0 = []
#     for q in robot0_eef_quat:
#         eef_rot6d0.append(quat_to_rot6d(q))
#     robot0_eef_rot6d = np.array(eef_rot6d0)
#     state = np.hstack([state, robot0_eef_rot6d])


#     repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
#     robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
#     robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
#     state = np.hstack([state, robot0_gripper_pos])

#     repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
#     robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
#     state = np.hstack([state, robot1_eef_pos])

#     repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
#     robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
#     eef_rot6d1 = []
#     for q in robot1_eef_quat:
#         eef_rot6d1.append(quat_to_rot6d(q))
#     robot1_eef_rot6d = np.array(eef_rot6d1)
#     state = np.hstack([state, robot1_eef_rot6d])

#     repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
#     robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
#     robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
#     state = np.hstack([state, robot1_gripper_pos])
    
# print(np.shape(state))
# print(np.shape(actions))

In [15]:
expert_states_list = []
expert_actions_list = []
pot_states_list = []
for i in [2, 3]:
    for j in range(10):   
        with open("rollouts_grippause/rollout_seed%s_mode%s.pkl" % (j*10, i), "rb") as f:
            rollout = pkl.load(f)
            obs = rollout["observations"]
            actions = np.array(rollout["actions"])
            pot = np.array(rollout["pot_pos"])

            pot_states_list.append(np.concatenate((pot[0], pot[1])))

            pos0 = actions[:,:3]
            rotvec0 = actions[:,3:6]
            gripper0 = actions[:,6]
            pos1 = actions[:,7:10]
            rotvec1 = actions[:,10:13]
            gripper1 = actions[:,13]

            rot6d_list0 = []
            for rv in rotvec0:
                rot6d_list0.append(rotvec_to_rot6d(rv))
            rot6d0 = np.array(rot6d_list0)

            rot6d_list1 = []
            for rv in rotvec1:
                rot6d_list1.append(rotvec_to_rot6d(rv))
            rot6d1 = np.array(rot6d_list1)

            actions = np.concatenate((pos0, rot6d0, gripper0.reshape(-1, 1), pos1, rot6d1, gripper1.reshape(-1, 1)), axis=1)

            robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
            robot0_eef_quat = np.array([o["robot0_eef_quat_site"] for o in obs])
            robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
            robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
            robot1_eef_quat = np.array([o["robot1_eef_quat_site"] for o in obs])
            robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

            repeats_needed = 400 - actions.shape[0]

            repeated_last = np.tile(actions[-1], (repeats_needed, 1))
            actions = np.vstack([actions, repeated_last])

            repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
            robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
            state = robot0_eef_pos

            repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
            robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
            eef_rot6d0 = []
            for q in robot0_eef_quat:
                eef_rot6d0.append(quat_to_rot6d(q))
            robot0_eef_rot6d = np.array(eef_rot6d0)
            state = np.hstack([state, robot0_eef_rot6d])


            repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
            robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
            robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
            state = np.hstack([state, robot0_gripper_pos])

            repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
            robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
            state = np.hstack([state, robot1_eef_pos])

            repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
            robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
            eef_rot6d1 = []
            for q in robot1_eef_quat:
                eef_rot6d1.append(quat_to_rot6d(q))
            robot1_eef_rot6d = np.array(eef_rot6d1)
            state = np.hstack([state, robot1_eef_rot6d])

            repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
            robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
            robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
            state = np.hstack([state, robot1_gripper_pos])

            expert_states_list.append(state)
            expert_actions_list.append(actions)

expert_states_rot6d = np.stack(expert_states_list, axis=0)
expert_actions_rot6d = np.stack(expert_actions_list, axis=0)
pot_states_rot6d = np.stack(pot_states_list, axis=0)

In [16]:
print(np.shape(expert_states_rot6d))
print(np.shape(expert_actions_rot6d))
print(np.shape(pot_states_rot6d))

(20, 400, 20)
(20, 400, 20)
(20, 6)


In [17]:
np.save("data/expert_states_rot6d_site_grippause_20.npy", expert_states_rot6d)
np.save("data/expert_actions_rot6d_site_grippause_20.npy", expert_actions_rot6d)
np.save("data/pot_states_rot6d_site_grippause_20.npy", pot_states_rot6d)